<h1> HaloPSA - API Module <h1>

In [ ]:
import re

from ts_api_connections import halopsa_api
from ts_api_connections import  datto_rmm_api

In [ ]:
halopsa = halopsa_api.HaloPSA()

In [ ]:
df_tickets = halopsa.create_tickets_dataframe()

In [ ]:
parser = halopsa_api.ParseSource()

In [ ]:
df_parsed = parser.add_parsed_info(df_tickets)

In [ ]:
df_datto_tickets = df_parsed[df_parsed['rootParse'] == 'centrastage']

In [ ]:
df_datto_tickets_open = df_datto_tickets[df_datto_tickets['hasbeenclosed'] == False]

In [ ]:
datto = datto_rmm_api.DattoRMM()

In [ ]:
df_alerts = datto.create_monitors_dataframe()

In [ ]:
df_alerts_open = df_alerts[df_alerts['resolved'] == True]

In [ ]:
df_disk_alerts = df_alerts_open[df_alerts_open['monitorSource'] == "Disk Usage High"]

In [ ]:
{'deviceUid': '587f7e09-00b4-e966-b4e9-dfa0ad689af8', 'deviceName': 'AWS-PROD-SQL1', 'siteUid': '[REDACTED_UUID]', 'siteName': 'Example'}

In [ ]:
df_disk_alerts

In [ ]:
df_datto_tickets.to_csv('.csv')

In [ ]:
df_alert_tickets['summary'].apply(lambda x: re.findall(r'Alert\{([\w\-]+)\}', str(x)))

In [ ]:
df_actions['datetime'] = pd.to_datetime(df_actions['datetime']).dt.strftime("%Y-%m-%d %H:%M:%S")
df_actions['actionarrivaldate'] = pd.to_datetime(df_actions['actionarrivaldate']).dt.strftime("%Y-%m-%d %H:%M:%S")
df_actions['actioncompletiondate'] = pd.to_datetime(df_actions['actioncompletiondate']).dt.strftime("%Y-%m-%d %H:%M:%S")

In [ ]:
df_actions

In [ ]:
df_tickets = halopsa.create_tickets_dataframe(days=4)

In [ ]:
df_users = halopsa.create_users_dataframe()

In [ ]:
df_dhavens = df_tickets[df_tickets.agent_id == 27]

In [ ]:
df_dhavens.ticketType.value_counts().plot(kind='bar')

In [ ]:
df_dhavens[df_dhavens.ticketType == 'Quick Time']

In [ ]:
df_dhavens

In [ ]:
useful_cols = ['id', 'dateoccurred', 'summary', 'details',
               'status_id', 'tickettype_id', 'sla_id',
               'priority_id', 'client_id', 'client_name',
               'site_id', 'site_name', 'user_id',
               'user_name', 'team', 'agent_id',
               'category_1', 'category_2', 'category_3', 'category_4',
               'child_count', 'attachment_count', 'flagged', 'read',
               'enduserstatus', 'dateclosed', 'lastactiondate',
               'department_id', 'workflow_step', 'workflow_seq',
               'hasbeenclosed', 'impact', 'urgency', 'ticketage',
               'closure_agent_id', 'closed_in_integration_system',
               'parent_id', 'dateassigned', 'reportedby', 'user_email', 'emailtolist',
               'emailcclist', 'inventory_number', 'ticketType', 'ticketStatus', 'reportCreationDate']


In [ ]:
df_dhavens = df_dhavens[useful_cols]

In [ ]:
df_dhavens

In [ ]:
parse = ParseSource()

In [ ]:
df_parsed = parse.add_parsed_info(df_tickets)

In [ ]:

class HaloPSADev:

    def __init__(self, env_file='d:/config/env.ini'):
        # import configparser for env secrets
        from configparser import ConfigParser

        config = ConfigParser()
        config.read(env_file)

        halo_config = config['halopsa']

        # HaloPSA Secrets Creation
        self.__base_uri = halo_config['base_uri']
        self.__client_id = halo_config['client_id']
        self.__client_secret = halo_config['client_secret']
        self.__tenant = halo_config['tenant']

        self.__access_token = self.__create_token()

    def __create_token(self):
        # call token api url
        token_uri = f"{self.__base_uri}/auth/token?tenant={self.__tenant}"

        # construct header
        headers = CaseInsensitiveDict()
        headers['Content-Type'] = 'application/x-www-form-urlencoded'

        # construct req body
        data = CaseInsensitiveDict()
        data['grant_type'] = 'client_credentials'
        data['client_id'] = self.__client_id
        data['client_secret'] = self.__client_secret
        data['scope'] = 'all'

        # request content response
        resp = requests.post(token_uri, headers=headers, data=data)
        content = resp.content.decode('utf-8')
        c_dict = json.loads(content)

        return c_dict


    def create_actions_dataframe(self,ticket_id):
        # request content response

        # https://halo.haloservicedesk.com/api/Actions?ticket_id=
        request_url = f"{self.__base_uri}/api/Actions?ticket_id={ticket_id}"

        # construct header
        headers = CaseInsensitiveDict()
        headers['Authorization'] = f'Bearer {self.__access_token}'
        headers['Content-Type'] = 'application/json'

        params = CaseInsensitiveDict()
        params = ""
        # params["ticket_id"] = ticket_id

        # construct req body
        data = ''

        print(f'Request URL: {request_url}')

        resp = requests.get(request_url, headers=headers, params=params, data=data)
        # content = resp.content.decode('utf-8')
        # c_dict = json.loads(content)

        print(resp.content)
        #
        # # iterate and combine remaining pages
        # df_actions = pd.DataFrame(c_dict['actions'])
        #
        # try:
        #   while c_dict['pageDetails']['nextPageUrl']:
        #       next_page = c_dict['pageDetails']['nextPageUrl']
        #       resp = requests.get(next_page, headers=headers, data=data)
        #       content = resp.content.decode('utf-8')
        #       c_dict = json.loads(content)
        #
        #       df_current_page = pd.DataFrame(c_dict['assets'])
        #       df_actions = pd.concat([df_actions, df_current_page], ignore_index=False)
        # except:
        #     print(f'All assets on first page.  Total Assets: {c_dict["record_count"]}')
        #
        #     report_creation_date = (dt.datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S'))
        #     df_actions['reportCreationDate'] = report_creation_date
        #
        # return df_actions






In [ ]:
ticket_id = 36158

In [ ]:
halo_test = HaloPSADev()

In [ ]:
df_appts = halo_test.create_actions_dataframe(ticket_id)

In [ ]:
df_appts

In [ ]:
df_appts[['attendees', 'attendees_list', 'appointment_origin']]

In [ ]:
df_appts[['attendees_list', 'appointment_origin']].iloc[1]['attendees_list'][0]

In [ ]:
df_appts['start_date'].min()

In [ ]:
df_appts['start_date'].max()

In [ ]:
df_appts['appointment_origin'].value_counts().plot(kind='bar')

In [ ]:
fig = plt.figure(figsize=(80, 40))
sns.barplot(data=df_appts, x='appointment_origin', y='appointment_duration')
fig.savefig('.png')

In [ ]:
df_appts.columns

In [ ]:
df_appts.subject.value_counts()

In [ ]:
fig = plt.figure(figsize=(80, 40))
sns.barplot(data=df_appts, x=df_appts.day_occurred, y='subject')